In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [49]:
MV = pd.read_csv('D:\Honzík\Škola\IES\V. Semestr\Data Processing in Python\Project\Market Values.csv')

In [50]:
PS = pd.read_csv('C:/Users/jancv/Final_Player_Table.csv')

In order to merge these two data frames by Players, we need to change the special Czech letters to international ones because the websites use diffetent encodings.

In [51]:
CZ = ["á", "č", "ď", "é", "ě", "í", "ň", "ó", "ř", "š", "ť", "ú", "ů", "ý", "ž", "Á", "Č", "Ď", "É", "Ě", "Í", "Ň", "Ó", "Ř", "Š", "Ť", "Ú", "Ů", "Ý", "Ž"]
NO = ["a","c","d","e","e","i","n","o","r","s","t","u","u","y","z","A","C","D","E","E","I","N","O","R","S","T","U","U","Y","Z"]
Player = []
for player in PS['player']:
    for idx in range(len(CZ)):
        player = player.replace(CZ[idx],NO[idx])
    Player.append(player)
PS['Player'] = Player

PS = PS.drop_duplicates('Player',keep=False)
PS = PS.drop(columns=['player'])

In [52]:
Dataset = pd.merge(PS,MV,on=['Player'],how='inner')
Dataset

,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,assists,...,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,Player,Foot,Market Value
0,CRO,MF,Viktoria Plzeň,26,1994,9,0,116,0,0,...,2,0,0.00,0.0,0.00,0.00,0.00,Marko Alvir,right,500
1,GHA,DF,Příbram,24,1996,8,2,273,2,0,...,0,0,0.66,0.0,0.66,0.66,0.66,Emmanuel Antwi,left,300
2,SVK,GK,Slovácko,29,1991,6,6,540,0,0,...,1,0,0.00,0.0,0.00,0.00,0.00,Pavol Bajza,right,200
3,SVK,MF,Zbrojovka Brno,26,1994,5,3,372,0,0,...,2,0,0.00,0.0,0.00,0.00,0.00,Damian Baris,right,300
4,CZE,MF,Bohemians 1905,32,1988,4,4,345,1,0,...,0,0,0.26,0.0,0.26,0.26,0.26,David Bartek,right,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,CZE,FW,Sigma Olomouc,21,1999,10,1,194,1,0,...,0,0,0.46,0.0,0.46,0.46,0.46,Pavel Zifcak,right,250
360,CZE,DF,Slavia Prague,20,2000,10,10,886,0,1,...,0,0,0.00,0.1,0.10,0.00,0.10,David Zima,-,3000
361,CZE,MF,Teplice,21,1999,11,9,809,1,0,...,3,0,0.11,0.0,0.11,0.11,0.11,Patrik Zitny,-,600
362,CZE,MF,Sigma Olomouc,21,1999,8,8,623,1,0,...,0,0,0.14,0.0,0.14,0.14,0.14,Ondrej Zmrzly,-,350


Now, we will add the current ranking in the Fortuna League by teams.

In [53]:
#by 1.1.2021
ranks = {'Slavia Prague':1,'Jablonec':2,'Sparta Prague':3,'Sigma Olomouc':4,'FK Pardubice':5,'Slovácko':6,'Slovan Liberec':7
                        ,'Baník Ostrava':8,'Karviná':9,'Viktoria Plzeň':10,'Teplice':11,'Fastav Zlín':12,'České Budĕjov.':13
                        ,'Bohemians 1905':14,'Příbram':15,'Zbrojovka Brno':16,'Mladá Boleslav':17,'Opava':18}

In [54]:
s=[]
for k in Dataset['squad']:
    s.append(ranks[k])
Dataset['Team_rank'] = s

And further, we will scrape the latest FIFA nationality ranking and add it to the Dataset using dictionary.

In [55]:
url = 'https://www.fifa.com/fifa-world-ranking/ranking-table/men/' #by 10.12.2020 (latest)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
players = soup.find("table",{"id": "rank-table"}).tbody.find_all('tr')

keys = []
values = []
for idx,player in enumerate(players):
    keys.append(player.find("span",{"class":"fi-t__nTri"}).text)
    values.append(idx+1)
Nation_ranking = dict(zip(keys,values))

In [56]:
nation = []
for nat in Dataset['nationality']:
    try:
        nation.append(int(Nation_ranking[nat]))
    except:
        nation.append(0)

Dataset['Nation_rank'] = nation
Dataset = Dataset[Dataset['Nation_rank']!=0]

#### Renaming the positions

Since there are some players who played on more than one position, we need to specify which is the dominant one to have only
3 levels. This is done in the following step.

In [57]:
pos2 = []
for p in Dataset['position']:
    if len(p) == 2:
        pos2.append(p)
    else:
        pos2.append(p[0:2])
Dataset['Pos2'] = pos2

In [58]:
d_pos = {'DF':'Defender','MF':'Midfielder','FW':'Forward'}
Position = []
Dataset = Dataset[Dataset['Pos2']!='GK']
for po in Dataset['Pos2']:
    Position.append(d_pos[po])
Dataset['Position'] = Position

In [59]:
Dataset = Dataset.drop(columns=['position','Pos2'])

In [60]:
Dataset.columns

Index(['nationality', 'squad', 'age', 'birth_year', 'games', 'games_starts',
       'minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'cards_yellow',
       'cards_red', 'goals_per90', 'assists_per90', 'goals_assists_per90',
       'goals_pens_per90', 'goals_assists_pens_per90', 'Player', 'Foot',
       'Market Value', 'Team_rank', 'Nation_rank', 'Position'],
      dtype='object')

### OLS

First of all, we would like to say that since we have a lot of variables and comparatively low amount of observations,
the OLS is not ideal. This project is, however, more of a data science one (i.e. getting the data, merging it etc.)
than econometric one (checking the validity of assumptions.)

In [61]:
import statsmodels.api as sm
import math

In [73]:
Y = Dataset['Market Value']
age = Dataset['age']
goals = Dataset['goals']
games = Dataset['games']
assists = Dataset['assists']
minutes = Dataset['minutes']
tr = Dataset['Team_rank']
nr = Dataset['Nation_rank']


X = np.column_stack((age,age^2,goals,assists,minutes,tr))
X = sm.add_constant(X)

model = sm.OLS(np.log(Y),X)
X = sm.add_constant(X)

In [74]:
fit = model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Market Value   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     32.93
Date:                Sun, 31 Jan 2021   Prob (F-statistic):           5.90e-31
Time:                        13:41:15   Log-Likelihood:                -339.44
No. Observations:                 332   AIC:                             692.9
Df Residuals:                     325   BIC:                             719.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.3645      0.222     28.674      0.000       5.928       6.801
x1            -0.0139      0.020     -0.682      0.496      -0.054       0.026
x2            -0.0036      0.019     -0.193      0.847      -0.041       0.033
x3             0.1389      0.029      4.732      0.000       0.081       0.197
x4             0.1133      0.044      2.562      0.011       0.026       0.200
x5             0.0006      0.000      5.169      0.000       0.000       0.001
x6            -0.0596      0.007     -8.177      0.000      -0.074      -0.045
==============================================================================
Omnibus:                        6.887   Durbin-Watson:                   1.854
Prob(Omnibus):                  0.032   Jarque-Bera (JB):                6.877
Skew:                           0.289   Prob(JB):                       0.0321
Kurtosis:                       3.403   Cond. No.                     4.02e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

As we can see the model behaves badly. If we were to collect more observations, it would definitely improve.
